## `[todo]` Title of project

### `[todo]` Reason for project (question)

### `[todo]` Hoping to achieve


### Importing and Checking Data

In [101]:
# importing libraries
import pandas as pd
import numpy as np
import re
import rich
from rich.console import Console

console = Console()
print = console.print

# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)  

# Reading data
url = 'https://raw.githubusercontent.com/Reece323/air-scrape/main/Data/Bentonville_Ar.csv'
df = pd.read_csv(url)
print(f'Local file: {url} loaded successfully.\n')

# Removing empty/unused columns and checking size of dataframe
df = df.drop(columns=['host_joined','house_rules', 'specialties_1', 'name'])
print('Empty/uneeded columns dropped...\n')

# checking shape
print(f"The dataset has {df.shape[0]} rows and {df.shape[1]} columns with {df.duplicated().sum()} duplicates.\n")

# Looking at data columns and head
names = df.columns.to_list()
print(f'Column names:\n{names}\n')

#dtypes
dataTypeSeries = df.dtypes.value_counts()
print(f'Data type per column: \n{dataTypeSeries}\n')

Local file: 
https://raw.githubusercontent.com/Reece323/air-scrape/main/Data/Bentonville_Ar.csv loaded 
successfully.

Empty/uneeded columns dropped...

The dataset has 300 rows and 14 columns with 0 duplicates.

Column names:
['url', 'header', 'guests', 'rooms', 'beds', 'baths', 'facilities', 'badge', 'rating', 
'review_count', 'price', 'sp_url', 'location', 'amenities']

Data type per column: 
object    14
dtype: int64

In [102]:
# Splitting header column
df['area'] = df['header'].replace(r' in ', '-' ,regex=True)
stucture = df['area'].str.split('-', n = 1, expand = True)

# adding new and dropping old subsets
df["listing_type"]= stucture[0]
df['location_specific'] = stucture[1]

# dropping old column
df.drop(columns =["header", 'location', 'area'], inplace = True)

In [103]:
print(f'listing_type: \n{df["listing_type"].value_counts(normalize=True).nlargest(3)}\n')
print(f'location_specific: \n{df["location_specific"].value_counts(normalize=True).nlargest(3)}\n')

listing_type: 
Entire residential home    0.650000
Entire townhouse           0.103333
Entire rental unit         0.043333
Name: listing_type, dtype: float64

location_specific: 
Fayetteville    0.286667
Bella Vista     0.260000
Bentonville     0.146667
Name: location_specific, dtype: float64

### Price, Guests, Rooms, Beds, Baths Columns: Raw value is *str*, prefer int/float dtypes for better use

In [104]:
df = df.rename(columns={'price': 'price_per_night'})
#changing price from string to int
df['price_per_night'] = df['price_per_night'].str.replace('$', '',regex=True).str.replace(',', '')
df['price_per_night'] = df['price_per_night'].astype(int)
print(f"The dataset has {df.shape[0]} rows and {df.shape[1]} columns with {df.duplicated().sum()} duplicates.\n")

The dataset has 300 rows and 14 columns with 0 duplicates.

In [105]:
#function to check dtypes and % of unique values
def checking(df_to_check):
    print(f"Cleaned data type: {df_to_check.dtype}")
    print(f'% of listings with # unique: \n{df_to_check.value_counts(normalize=True).nlargest(4)}\n')

In [106]:
# 'guests'
new = df["guests"].str.split(" ", n = 1, expand = True)
# making separate column from new data frame
df["guests"] = new[0]
# checking 'guests'
checking(df.guests)

# 'rooms'
df['rooms'] = df['rooms'].replace('Studio', '0 bedrooms')
new = df["rooms"].str.split(" ", n = 1, expand = True)
# making separate column from new data frame
df["rooms"] = new[0]
# changing str to int
df['rooms'] = df['rooms'].astype(int) 
# checking 'rooms'
checking(df.rooms)

# 'beds'
df['beds'] = df['beds'].str.replace(r'2.5 baths', '3 beds',regex=True)
new = df["beds"].str.split(" ", n = 1, expand = True)
# making separate column from new data frame
df["beds"] = new[0]
# changing str to int
df['beds'] = df['beds'].astype(int) 
# checking 'beds'
checking(df.beds)

# 'baths'
df['baths'] = df['baths'].str.replace(r'Wifi', '2.5 baths',regex=True)
df['baths'] = df['baths'].str.replace(r'Half-bath', '.5 baths',regex=True)
new = df["baths"].str.split(" ", n = 1, expand = True)
# making separate column from new data frame
df["baths"] = new[0]
# changing str to int
df['baths'] = df['baths'].astype(float) 
# checking 'baths'
checking(df.baths)

Cleaned data type: object

% of listings with # unique: 
6     0.306667
8     0.216667
4     0.180000
10    0.066667
Name: guests, dtype: float64

Cleaned data type: int64

% of listings with # unique: 
3    0.430000
2    0.240000
4    0.170000
1    0.063333
Name: rooms, dtype: float64

Cleaned data type: int64

% of listings with # unique: 
3    0.270000
4    0.243333
2    0.143333
5    0.136667
Name: beds, dtype: float64

Cleaned data type: float64

% of listings with # unique: 
2.0    0.456667
2.5    0.140000
1.0    0.140000
3.0    0.110000
Name: baths, dtype: float64

### Column Facilities: Wanting to split and have their own columns

In [107]:
LIST_FACILITIES = [
    'Kitchen',
    'Wifi',
    'Free parking',
    'Self check-in'
]

def make_facilities(data, list_facilities=LIST_FACILITIES):
    data_facilities = data.loc[:, ['url', 'facilities']]

    for i in list_facilities:
        data_facilities[f"facility_{i}"] = data_facilities['facilities'].fillna('empty').str.split(' · ').apply(lambda x: 1 if i in x else 0)
    data_facilities.drop('facilities', axis=1, inplace=True)

    return data_facilities

# Merging new facilities
df_facilities = make_facilities(df)
clean_df = pd.merge (df_facilities, df)
clean_df = clean_df.drop(columns=['facilities'])

### Working on *badge*, *reviews*, and *rating* columns

In [108]:
# working on 'badge' column
clean_df = clean_df.rename(columns={'badge': 'superhost'})
clean_df["superhost"].replace({'SUPERHOST': 1, 'empty': 0}, inplace=True)

In [109]:
# 'review_count' column
clean_df['review_count'] = clean_df['review_count'].str.replace(r'(', '',regex=True)
clean_df['review_count'] = clean_df['review_count'].str.replace(r')', '',regex=True)
new = clean_df['review_count'].str.replace(r'empty', ' 0 reviews',regex=True)
new = new.replace(r'\D', '',regex=True)
clean_df['review_count'] = new


In [110]:
# 'rating' column
clean_df['rating'] = clean_df['rating'].str.replace(r'empty', 'none',regex=True)
clean_df.head()


,url,facility_Kitchen,facility_Wifi,facility_Free parking,facility_Self check-in,guests,rooms,beds,baths,superhost,rating,review_count,price_per_night,sp_url,amenities,listing_type,location_specific
0,/rooms/568774443413437016?adults=4&check_in=20...,1,1,1,1,8,3,4,2.0,1,none,0,126,https://www.airbnb.com/s/Bentonville--Arkansas...,"{""amenities"": [""Hair dryer"", ""Cleaning product...",Entire residential home,Downtown Bentonville
1,/rooms/39864150?adults=4&check_in=2022-03-30&c...,1,1,1,1,8,4,4,3.0,0,none,0,121,https://www.airbnb.com/s/Bentonville--Arkansas...,"{""amenities"": [""Bathtub"", ""Hair dryer"", ""Washe...",Entire residential home,Bentonville
2,/rooms/23161645?adults=4&check_in=2022-03-30&c...,1,1,1,1,8,3,3,2.5,1,4.99,101,194,https://www.airbnb.com/s/Bentonville--Arkansas...,"{""amenities"": [""Bathtub"", ""Hair dryer"", ""Shamp...",Entire townhouse,Downtown Bentonville
3,/rooms/584082363006663396?adults=4&check_in=20...,1,1,1,1,6,3,3,2.0,1,none,0,122,https://www.airbnb.com/s/Bentonville--Arkansas...,"{""amenities"": [""Washer"", ""TV"", ""Air conditioni...",Entire residential home,Bella Vista
4,/rooms/31653766?adults=4&check_in=2022-03-30&c...,1,1,1,1,6,3,3,2.0,1,4.90,31,129,https://www.airbnb.com/s/Bentonville--Arkansas...,"{""amenities"": [""Hair dryer"", ""Shampoo"", ""Hot w...",Entire residential home,Bentonville


### Column *listing type*

In [111]:
LISTING_TYPES = [
    'Entire residential home', 'Entire townhouse',
    'Entire condominium (condo)', 'Entire cottage',
    'Entire rental unit', 'Entire loft', 'Entire guesthouse',
    'Entire cabin', 'Tiny house', 'Entire vacation home',
    'Resort room', 'Entire place', 'Barn', 'Entire guest suite',
    'Private room', 'Camper/RV', 'Entire bungalow', 'Entire villa',
    'Campsite', 'Entire chalet', 'Farm stay'
]

def make_listings(data, listing_types=LISTING_TYPES):
    data_listing_type = data.loc[:, ['url', 'listing_type']]

    for i in listing_types:
        data_listing_type[f"type_{i}"] = data_listing_type['listing_type'].fillna('empty').str.split(' · ').apply(lambda x: 1 if i in x else 0)
    data_listing_type.drop('listing_type', axis=1, inplace=True)

    return data_listing_type

# Merging new facilities
df_listing_types = make_listings(clean_df)
df = pd.merge (df_listing_types, clean_df)
df = df.drop(columns=['listing_type'])
df.columns = df.columns.str.replace(' ', '_')

### Column *location_specific*

In [112]:
Location_Specific = ['Downtown Bentonville', 'Bentonville', 'Bella Vista', 'Rogers',
       'Centerton', 'Fayetteville', 'Springdale', 'Lowell', 'Township 2',
       'Little Flock', 'Washington County', 'Farmington'
]

def make_listings(data, location=Location_Specific):
    location_type = data.loc[:, ['url', 'location_specific']]

    for i in location:
        location_type[f"area_{i}"] = location_type['location_specific'].fillna('empty').str.split(' · ').apply(lambda x: 1 if i in x else 0)
    location_type.drop('location_specific', axis=1, inplace=True)

    return location_type

# Merging new locations
df_listing_types = make_listings(df)
df = pd.merge (df_listing_types, df)
df = df.drop(columns=['location_specific'])
df.columns = df.columns.str.replace(' ', '_')

In [113]:
df.rename(columns={'type_Entire_condominium_(condo)': 'type_Entire_condominium'}, inplace = True)

In [114]:
print(df.columns)

Index(['url', 'area_Downtown_Bentonville', 'area_Bentonville',
       'area_Bella_Vista', 'area_Rogers', 'area_Centerton',
       'area_Fayetteville', 'area_Springdale', 'area_Lowell',
       'area_Township_2', 'area_Little_Flock', 'area_Washington_County',
       'area_Farmington', 'type_Entire_residential_home',
       'type_Entire_townhouse', 'type_Entire_condominium',
       'type_Entire_cottage', 'type_Entire_rental_unit', 'type_Entire_loft',
       'type_Entire_guesthouse', 'type_Entire_cabin', 'type_Tiny_house',
       'type_Entire_vacation_home', 'type_Resort_room', 'type_Entire_place',
       'type_Barn', 'type_Entire_guest_suite', 'type_Private_room',
       'type_Camper/RV', 'type_Entire_bungalow', 'type_Entire_villa',
       'type_Campsite', 'type_Entire_chalet', 'type_Farm_stay',
       'facility_Kitchen', 'facility_Wifi', 'facility_Free_parking',
       'facility_Self_check-in', 'guests', 'rooms', 'beds', 'baths',
       'superhost', 'rating', 'review_count', 'price_per_night', 'sp_url',
       'amenities'],
      dtype='object')

In [115]:
#reordering columns to have urls up front
cols = list(df)
# print(cols)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('sp_url')))
# print(cols)
# use ix to reorder
df = df.loc[:, cols]

In [116]:
df[['area_Downtown_Bentonville', 'area_Bentonville', 'area_Bella_Vista',
       'area_Rogers', 'area_Centerton', 'area_Fayetteville', 'area_Springdale',
       'area_Lowell', 'area_Township_2', 'area_Little_Flock',
       'area_Washington_County', 'area_Farmington',
       'type_Entire_residential_home', 'type_Entire_townhouse',
       'type_Entire_condominium',
       'type_Entire_cottage', 'type_Entire_rental_unit',
       'type_Entire_loft', 'type_Entire_guesthouse',
       'type_Entire_cabin', 'type_Tiny_house',
       'type_Entire_vacation_home', 'type_Resort_room',
       'type_Entire_place', 'type_Barn',
       'type_Entire_guest_suite', 'type_Private_room',
       'type_Camper/RV', 'type_Entire_bungalow',
       'type_Entire_villa', 'type_Campsite',
       'type_Entire_chalet', 'type_Farm_stay',
       'facility_Kitchen', 'facility_Wifi', 'facility_Free_parking',
       'facility_Self_check-in', 'guests', 'rooms', 'beds', 'baths',
       'superhost', 'rating', 'review_count', 'price_per_night']].head()

,area_Downtown_Bentonville,area_Bentonville,area_Bella_Vista,area_Rogers,area_Centerton,area_Fayetteville,area_Springdale,area_Lowell,area_Township_2,area_Little_Flock,area_Washington_County,area_Farmington,type_Entire_residential_home,type_Entire_townhouse,type_Entire_condominium,type_Entire_cottage,type_Entire_rental_unit,type_Entire_loft,type_Entire_guesthouse,type_Entire_cabin,type_Tiny_house,type_Entire_vacation_home,type_Resort_room,type_Entire_place,type_Barn,type_Entire_guest_suite,type_Private_room,type_Camper/RV,type_Entire_bungalow,type_Entire_villa,type_Campsite,type_Entire_chalet,type_Farm_stay,facility_Kitchen,facility_Wifi,facility_Free_parking,facility_Self_check-in,guests,rooms,beds,baths,superhost,rating,review_count,price_per_night
0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,8,3,4,2.0,1,none,0,126
1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,8,4,4,3.0,0,none,0,121
2,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,8,3,3,2.5,1,4.99,101,194
3,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,6,3,3,2.0,1,none,0,122
4,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,6,3,3,2.0,1,4.90,31,129


### Tackling amenities

In [118]:
df.loc[df['amenities'].str.contains('tv', case=False), 'amenities'] = 'TV'

In [ ]:
df['amenities'].value_counts(normalize=True)